# Binomial Tree

$B_k$ is formed by attatching a binomial tree $B_{k-1}$

# Binomial Heap

    # is binomial tree with heap property

$6_{10} = 110_2$

For exapmple if I want 6 nodes, I need one of $B_1$ and $B_2$ / if I want 14 nodes, I need one of $B_4$ and $B_2$

This likes Binary number

In [ ]:
import math

def MakeBinomialHeap():
    class BinomialHeap:
        def __init__(self):
            self.head = None
    return BinomialHeap()

def MergeHeap(H1, H2):
    if H1.head is None:
        return H2.head
    if H2.head is None:
        return H1.head

    head = None
    tail = None

    h1_curr = H1.head
    h2_curr = H2.head

    # Initialize head
    if h1_curr.degree <= h2_curr.degree:
        head = h1_curr
        h1_curr = h1_curr.sibling
    else:
        head = h2_curr
        h2_curr = h2_curr.sibling

    tail = head

    # Merge the two root lists by degree
    while h1_curr and h2_curr:
        if h1_curr.degree <= h2_curr.degree:
            tail.sibling = h1_curr
            h1_curr = h1_curr.sibling
        else:
            tail.sibling = h2_curr
            h2_curr = h2_curr.sibling
        tail = tail.sibling

    # Attach the remaining roots
    tail.sibling = h1_curr if h1_curr else h2_curr

    return head

In [ ]:
def FindMin(H):
    """
    takes about O(log n) time complexity
    """
    y = None
    x = H.head
    min_key = float('inf')
    while x is not None:
        if x.key < min_key:
            min_key = x.key
            y = x
        x = x.sibling
    return y


def combineBinomialTrees(t1, t2):
    """
    takes about O(1) time complexity
    """
    t1.parent = t2
    t1.sibling = t2.child
    t1.child = t2
    t2.degree = t2.degree + 1
    return t1

#Just Binary addition
def BinomialHeapUnion(H1, H2):
    """
    takes about O(log n) time complexity
    """
    H = MakeBinomialHeap()
    H.head = MergeHeap(H1, H2)
    if H.head is None:
        return H

    prev = None
    x = H.head
    next = x.sibling

    while next is not None:
        if (x.degree != next.degree) or (next.sibling is not None and next.sibling.degree == x.degree):
            prev = x
            x = next
        else:
            if x.key <= next.key:
                x.sibling = next.sibling
                combineBinomialTrees(next, x)
            else:
                if prev is None:
                    H.head = next
                else:
                    prev.sibling = next
                combineBinomialTrees(x, next)
                x = next
        next = x.sibling

    return H

def BinomialHeapInsert(H, x):
    """
    takes about O(log n) time complexity
    """
    H_prime = MakeBinomialHeap()
    x.parent = None
    x.child = None
    x.sibling = None
    x.degree = 0
    H_prime.head = x
    H = BinomialHeapUnion(H, H_prime)
    return H

def BinomialHeapExtractMin(H):
    """
    takes about O(log n) time complexity
    """
    if H.head is None:
        return None

    # Find the root with minimum key in root list
    prev_min = None
    min_node = H.head
    prev = None
    curr = H.head
    min_key = curr.key

    while curr is not None:
        if curr.key < min_key:
            min_key = curr.key
            prev_min = prev
            min_node = curr
        prev = curr
        curr = curr.sibling

    # Remove min_node from root list
    if prev_min is None:
        # min_node is head
        H.head = min_node.sibling
    else:
        prev_min.sibling = min_node.sibling

    # Reverse the order of min_node's children and set their parent to None
    child = min_node.child
    prev_child = None
    while child is not None:
        next_child = child.sibling
        child.sibling = prev_child
        child.parent = None
        prev_child = child
        child = next_child

    # Create new heap H' with head pointing to reversed children list
    H_prime = MakeBinomialHeap()
    H_prime.head = prev_child

    # Union H and H_prime
    H = BinomialHeapUnion(H, H_prime)

    return min_node

def BinomialHeapDecreaseKey(H, x, k):
    """
    takes about O(log n) time complexity
    """
    x.key = x.key - k
    y = x
    z = y.parent

    #percolade uo process
    while z is not None and y.key < z.key:
        # Swap keys
        y.key, z.key = z.key, y.key
        y = z
        z = y.parent

# Fibonacci Heaps

    # lazy binomial heap, allow to have duplicate degree.

we can cut at most one child.

if cut another child, the node needs to be cut from its parent and become the root of new tree.

In [ ]:
class Node:
    def __init__(self, key):
        self.key = key
        self.degree = 0
        self.parent = None
        self.child = None
        self.mark = False   # represent the color
        self.left = self
        self.right = self

class FibHeapInsert:
    """
    Takes O(1)
    """
    def __init__(self):
        self.min = None
        self.n = 0
        self.root_list = []

    def insert(self, x):
        x.degree = 0
        x.parent = None
        x.child = None
        x.mark = False

        #is new node
        if self.min is None:
            self.root_list = [x]
            self.min = x
        else:
            self.root_list.append(x)
            if x.key < self.min.key:
                self.min = x

        self.n += 1

def fib_heap_union(H1, H2):
    """
    Takes O(1)
    """
    H = FibHeapInsert()  # Create a new empty Fibonacci heap
    H.min = H1.min

    # Concatenate the root lists of H1 and H2
    H.root_list = H1.root_list + H2.root_list

    # Determine the new minimum
    if (H1.min is None) or (H2.min is not None and H2.min.key < H1.min.key):
        H.min = H2.min

    # Update the total number of nodes
    H.n = H1.n + H2.n

    return H

# ALGORITHM 21
def consolidate(H):
    """
    re-arrange the binomial tree to become
    the beautiful binomial tree
    Takes O(log n)

    Find the minimum value point the pointer.z to that minimum value
    then use that pointer.z walking to the right of the nodes
    create the array A[4] = (0, 1, 2, 3) to store each node with same degree as index
    A0 point to the node that has degree 0
    A1 point to the node that has degree 1
    A2 point to the node that has degree 2
    A3 point to the node that has degree 3
    repeating until we encounter the node that has a degree [i]
    which Array A[i] is not NULL (have a node with same degree)
    after we encounter that node we combine that node together
    then keep checking if A[i+1] is NULL or not, if not keep combine
    """
    # Upper bound on degree
    D = int(math.log2(H.n)) + 1
    A = [None] * D

    # Copy root list nodes to avoid modification during iteration
    root_nodes = list(H.root_list)
    for w in root_nodes:
        x = w
        d = x.degree
        while A[d] is not None:
            y = A[d]
            if x.key > y.key:
                x, y = y, x  # exchange x with y
            link(H, y, x)
            A[d] = None
            d += 1
        A[d] = x

    # Reconstruct root list and reset H.min
    H.root_list = []
    H.min = None
    for i in range(D):
        if A[i] is not None:
            H.root_list.append(A[i])
            if H.min is None or A[i].key < H.min.key:
                H.min = A[i]

def link(H, y, x):
    """Make y a child of x."""
    H.root_list.remove(y)
    if x.child is None:
        x.child = y
        y.left = y.right = y
    else:
        # Insert y into child list of x
        child = x.child
        y.left = child
        y.right = child.right
        child.right.left = y
        child.right = y
    y.parent = x
    x.degree += 1
    y.mark = False

# ALGORITHM 20
def fib_heap_extract_min(H):
    z = H.min
    if z is not None:
        # Add each child of z to the root list
        if z.child:
            children = []
            x = z.child
            while True:
                children.append(x)
                x = x.right
                if x == z.child:
                    break
            for x in children:
                x.parent = None
                H.root_list.append(x)

        # Remove z from the root list
        H.root_list.remove(z)

        # Update min pointer
        if not H.root_list:
            H.min = None
        else:
            H.min = H.root_list[0]
            """
            re-arrange the binomial tree to become
            the beautiful binomial tree
            Takes O(log n)
            """
            consolidate(H)

        H.n -= 1
    return z